<a href="https://colab.research.google.com/github/dizapl/Machine_Learning/blob/main/Lab10/%D0%97%D0%B0%D0%BF%D0%BB%D0%B5%D1%82%D0%BD%D1%8E%D0%BA_%D0%BF%D1%80_10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10
from tensorflow.image import resize

In [2]:
# Завантаження та підготовка даних
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Перетворення даних у формат float32 і нормалізація
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test = tf.cast(x_test, tf.float32) / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [9]:
# Завантаження DenseNet121 без верхніх шарів
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Заморожування всіх шарів базової моделі
for layer in base_model.layers:
    layer.trainable = False

# Створення моделі
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

In [4]:
# Компіляція моделі
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  # Зміна на 'categorical_crossentropy' для one-hot міток
              metrics=['accuracy'])

# Виведення структури моделі
model.summary()

ValueError: Undefined shapes are not supported.

In [ ]:
# Колбеки для ранньої зупинки
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Перетворення міток у формат one-hot для 'categorical_crossentropy'
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Тренування моделі
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/50
 884/1563 ━━━━━━━━━━━━━━━━━━━━ 1:24 124ms/step - accuracy: 0.4400 - loss: 1.8077

In [ ]:
# Візуалізація функцій втрат та точності
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Тренувальна точність')
plt.plot(history.history['val_accuracy'], label='Валізаційна точність')
plt.title('Точність моделі')
plt.xlabel('Епохи')
plt.ylabel('Точність')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Тренувальні втрати')
plt.plot(history.history['val_loss'], label='Валізаційні втрати')
plt.title('Втрати моделі')
plt.xlabel('Епохи')
plt.ylabel('Втрати')
plt.legend()

plt.show()

In [ ]:
# Класифікаційний звіт
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Тестова точність:', test_acc)

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred_classes))

In [ ]:
# Візуалізація розпізнавання
num_samples = 10
indices = np.random.choice(len(x_test), num_samples, replace=False)

plt.figure(figsize=(12, 6))

for i, index in enumerate(indices):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_test[index])
    plt.title(f'Прогноз: {y_pred_classes[index]},\nСправжній: {y_true[index]}')
    plt.axis('off')

plt.tight_layout()
plt.show()

**Висновок:** заморожування шарів базової моделі (як-от DenseNet121) і тренування лише додаткових шарів дозволяє застосувати знання, набуті на великих наборах даних, до нових завдань (як-от класифікація CIFAR-10), що підвищує ефективність і точність, особливо якщо тренувальний набір даних обмежений.